<a href="https://colab.research.google.com/github/Aditi2512Gupta/Aditi-Gupta/blob/main/Smart_Bus_Routing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import heapq, random

# ------------- GRAPH + DIJKSTRA ----------------
graph = {
    "A": [("B",3),("D",6),("E",2)],
    "B": [("A",3),("C",4)],
    "C": [("B",4),("D",5),("F",3)],
    "D": [("C",5),("A",6)],
    "E": [("A",2),("F",4)],
    "F": [("C",3),("E",4)]
}

def dijkstra(start):
    dist = {n: float("inf") for n in graph}
    dist[start] = 0
    pq = [(0,start)]
    while pq:
        d,u = heapq.heappop(pq)
        if d > dist[u]: continue
        for v,w in graph[u]:
            if d + w < dist[v]:
                dist[v] = d + w
                heapq.heappush(pq,(dist[v],v))
    return dist

# ------------- GENERATE REQUESTS ---------------
def generate_requests(n=30):
    nodes = list(graph.keys())
    reqs = []
    for t in range(n):
        s,d = random.sample(nodes,2)
        reqs.append([t*2, s, d])   # time, src, dst
    return reqs

# ------------- FIXED ROUTE BUS -----------------
def run_fixed(requests):
    route = ["A","B","C","D","A"]
    loc = "A"; wait = 0
    for t in range(200):
        for r in requests:
            if r[0] <= t and r[1] == loc and r[0] != -1:
                wait += t - r[0]
                r[0] = -1  # served
        loc = route[(route.index(loc)+1)%len(route)]
    return wait

# ------------- AGENT DIJKSTRA BUS --------------
def run_agent(requests):
    loc = "A"; wait = 0
    for t in range(200):
        # pick if same node
        for r in requests:
            if r[0] <= t and r[1] == loc and r[0] != -1:
                wait += t - r[0]
                r[0] = -1

        # find nearest request
        active = [r for r in requests if r[0] != -1 and r[0] <= t]
        if active:
            dist = dijkstra(loc)
            target = min(active, key=lambda x: dist[x[1]])[1]

            # move to neighbor closer to target
            dist2 = dijkstra(target)
            loc = min(graph[loc], key=lambda x: dist2[x[0]])[0]

    return wait

# ---------------- MAIN -------------------------
reqs1 = generate_requests()
reqs2 = [r.copy() for r in reqs1]

base = run_fixed(reqs1)
agent = run_agent(reqs2)
improve = (base - agent)/base * 100

print("Fixed-route wait:", base)
print("Agent-route wait:", agent)
print("Wait time improvement: {:.2f}%".format(improve))

Fixed-route wait: 37
Agent-route wait: 31
Wait time improvement: 16.22%
